# Transfer Learning
### Sentiment Analysis using BERT + Hugging Face

## What is Sentiment Analysis

# What is BERT?

<img src="./img/lab_12_bert.jpg">

**BERT** stands for Bidirectional Encoder Representations from Transformers. Jacob Devlin and his colleagues developed BERT at Google in 2018. Devlin and his colleagues trained the BERT on English Wikipedia (2,500M words) and BooksCorpus (800M words) and achieved the best accuracies for some of the NLP tasks in 2018. 


There are two pre-trained general BERT variations: The base model is a 12-layer, 768-hidden, 12-heads, 110M parameter neural network architecture, whereas the large model is a 24-layer, 1024-hidden, 16-heads, 340M parameter neural network architecture.

# What is Hugging Face 🤗

Hugging Face is an open-source provider of natural language processing (NLP) technologies. It has a large open-source community, in particular around the Transformers library.

🤗/Transformers is a python-based library that exposes an API to use many well-known transformer architectures, such as BERT, RoBERTa, GPT-2 or DistilBERT, that obtain state-of-the-art results on a variety of NLP tasks like text classification, information extraction, question answering, and text generation. Those architectures come pre-trained with several sets of weights. Getting started with Transformers only requires to install the pip package:

`pip install transformers`


more here: https://blog.tensorflow.org/2019/11/hugging-face-state-of-art-natural.html


# Sentiment Analysis using BERT

The advantage of using `Transformers` lies in the straight-forward model-agnostic API. Loading a pre-trained model, along with its tokenizer can be done in a few lines of code. Here is an example of loading the BERT TensorFlow models as well as their tokenizers:

In [5]:
from transformers import TFBertModel, BertTokenizer

bert_model = TFBertModel.from_pretrained("bert-base-cased")  # Automatically loads the config
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: 
TFBertModel requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.


## Fine-tuning a Transformer model